In [32]:
import pandas as pd
from beer_consumption.domain import preprocessing

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
filename = ("./beer_reviews.csv")
df = pd.read_csv(filename, sep=",")
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [34]:
cols = ["review_time", "beer_beerid"]
df[cols].duplicated().sum()

47

In [35]:
mask = df["review_time"] == 1269285610
df[mask]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
59970,16866,The Bruery,1269285610,4.5,4.0,3.5,alpinebryant,Witbier,3.5,4.5,Orchard White,5.7,42433
59971,16866,The Bruery,1269285610,4.0,3.5,4.0,BeerGodz,Witbier,4.0,3.5,Orchard White,5.7,42433


In [36]:
df.shape

(1586614, 13)

In [20]:
# brewery which produce the strongest beer
b = preprocessing.strongest_brewery(df)
{'brewery_name':b.to_dict()['brewery_name'].values()}

{'brewery_name': dict_values(['Schorschbräu'])}

there are two brewery_id with no corresponding brewery_name.
So in the following, we consider only brewery_id for analysis.

To compute beer name to recommend, we focus only on beers which where noted at least one time after 2010.

In [112]:
df = preprocessing.convert_time(df)
df = preprocessing.beer_after_date(df, 2011)

In [89]:
df.shape

(882358, 15)

In [90]:
len(pd.unique(df.beer_beerid))

4367

In [106]:
most_beer = preprocessing.top_most_beers(df) 

In [107]:
most_beer

[77303, 71913, 77128]

In [96]:
df.columns

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv', 'beer_beerid',
       'date', 'year'],
      dtype='object')

In [99]:
column_note = ['review_overall','review_aroma', 'review_appearance','review_palate','review_taste']
x = df[column_note[1:]]
y = df[column_note[0]]
df[column_note].corr()

,review_overall,review_aroma,review_appearance,review_palate,review_taste
review_overall,1.000000,0.588001,0.481835,0.678185,0.767719
review_aroma,0.588001,1.000000,0.550420,0.602191,0.702118
review_appearance,0.481835,0.550420,1.000000,0.556734,0.535293
review_palate,0.678185,0.602191,0.556734,1.000000,0.719946
review_taste,0.767719,0.702118,0.535293,0.719946,1.000000


la colonne la plus correlée est review_taste avec un coefficient de 0.767719 en prenant en compte la base sur l'année 2012

In [101]:
#pour la regression lineaire 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x,y)
model.coef_

ModuleNotFoundError: No module named 'sklearn'

In [119]:
#top de beer en fonction de l'apparence et arome

most_beer_apparence_arome, data = preprocessing.top_most_beers_apparence_aroma(df) 

data[data.index.isin(most_beer_apparence_arome)]


,review_overall,review_aroma,review_palate,review_taste,review_appearance
beer_beerid,,,,,
77128,5.0,4.00,4.000,5.00,5.0
77045,4.5,5.00,5.000,5.00,3.0
76989,4.5,5.00,4.500,4.50,4.5
77203,4.5,5.00,4.500,4.50,4.5
44605,4.5,4.75,4.875,4.75,5.0
76947,4.5,4.50,4.000,4.50,5.0
77093,4.5,3.50,5.000,4.00,5.0
77205,4.0,5.00,4.500,3.50,3.0
77071,4.0,5.00,3.500,4.00,4.0


In [29]:
df.shape

(1586614, 15)

In [26]:
len(beer_id_2010)

29824

In [27]:
df["beer_beerid"].nunique()

66055

In [5]:
df["brewery_id"].nunique()

5840

In [6]:
df["brewery_name"].nunique()

5742

In [13]:
mask = df["brewery_name"].isna()
df_na_name = df[mask]

In [16]:
df_na_name["brewery_id"].unique()

array([1193,   27])

In [18]:
df_na_name["brewery_id"].nunique()

2

In [4]:
mask = df["beer_abv"] == max(df["beer_abv"].unique())

In [5]:
df.loc[mask, ["brewery_id", "brewery_name", "beer_name", "beer_abv"]]

,brewery_id,brewery_name,beer_name,beer_abv
12919,6513,Schorschbräu,Schorschbräu Schorschbock 57%,57.7


In [9]:
df.isnull().sum().sort_values(ascending=False)

beer_abv              67785
review_profilename      348
brewery_name             15
beer_beerid               0
beer_name                 0
review_taste              0
review_palate             0
beer_style                0
review_appearance         0
review_aroma              0
review_overall            0
review_time               0
brewery_id                0
dtype: int64